In [10]:
%load_ext autoreload
%autoreload 2

# Benchmark

In [ ]:
import optimagic as om
problems = om.get_benchmark_problems("example")
a= om.algorithms.EnsmallenLBFGS
b = om.algos.scipy_cobyla
optimizers = {
    a,
    b,
}
results = om.run_benchmark(
    problems, 
    optimizers
)
# a = list(results.keys())[0]
# results[a]

fig = om.profile_plot(
    problems = problems,
    results=results,
)
fig.show()

# Ensmallen Module

In [4]:
"""Implement ensmallen optimizers."""

from dataclasses import dataclass

import numpy as np
from numpy.typing import NDArray

from optimagic import mark
from optimagic.config import IS_PYENSMALLEN_INSTALLED
from optimagic.optimization.algo_options import (
    CONVERGENCE_FTOL_REL,
    CONVERGENCE_GTOL_ABS,
    MAX_LINE_SEARCH_STEPS,
    STOPPING_MAXITER,
)
from optimagic.optimization.algorithm import Algorithm, InternalOptimizeResult
from optimagic.optimization.internal_optimization_problem import (
    InternalOptimizationProblem,
)
from optimagic.typing import AggregationLevel, NonNegativeFloat, PositiveInt

if IS_PYENSMALLEN_INSTALLED:
    import pyensmallen as pye

LIMITED_MEMORY_MAX_HISTORY = 10
"""Number of memory points to be stored (default 10)."""
MIN_LINE_SEARCH_STEPS = 1e-20
"""The minimum step of the line search."""
MAX_LINE_SEARCH_TRIALS = 50
"""The maximum number of trials for the line search (before giving up)."""
ARMIJO_CONSTANT = 1e-4
"""Controls the accuracy of the line search routine for determining the Armijo
condition."""
WOLFE_CONDITION = 0.9
"""Parameter for detecting the Wolfe condition."""


@mark.minimizer(
    name="ensmallen_lbfgs",
    solver_type=AggregationLevel.SCALAR,
    is_available=IS_PYENSMALLEN_INSTALLED,
    is_global=False,
    needs_jac=True,
    needs_hess=False,
    supports_parallelism=False,
    supports_bounds=False,
    supports_linear_constraints=False,
    supports_nonlinear_constraints=False,
    disable_history=False,
)
@dataclass(frozen=True)
class EnsmallenLBFGS(Algorithm):
    limited_memory_max_history: PositiveInt = LIMITED_MEMORY_MAX_HISTORY
    stopping_maxiter: PositiveInt = STOPPING_MAXITER
    armijo_constant: NonNegativeFloat = ARMIJO_CONSTANT  # needs review
    wolfe_condition: NonNegativeFloat = WOLFE_CONDITION  # needs review
    convergence_gtol_abs: NonNegativeFloat = CONVERGENCE_GTOL_ABS
    convergence_ftol_rel: NonNegativeFloat = CONVERGENCE_FTOL_REL
    max_line_search_trials: PositiveInt = MAX_LINE_SEARCH_TRIALS
    min_step_for_line_search: NonNegativeFloat = MIN_LINE_SEARCH_STEPS
    max_step_for_line_search: NonNegativeFloat = MAX_LINE_SEARCH_STEPS

    def _solve_internal_problem(
        self, problem: InternalOptimizationProblem, x0: NDArray[np.float64]
    ) -> InternalOptimizeResult:
        optimizer = pye.L_BFGS(
            numBasis=self.limited_memory_max_history,
            maxIterations=self.stopping_maxiter,
            armijoConstant=self.armijo_constant,
            wolfe=self.wolfe_condition,
            minGradientNorm=self.convergence_gtol_abs,
            factr=self.convergence_ftol_rel,
            maxLineSearchTrials=self.max_line_search_trials,
            minStep=self.min_step_for_line_search,
            maxStep=self.max_step_for_line_search,
        )

        def objective_function(
            x: NDArray[np.float64], grad: NDArray[np.float64]
        ) -> np.float64:
            grad[:] = problem.jac(x)
            return np.float64(problem.fun(x))

        raw = optimizer.optimize(objective_function, x0)

        res = InternalOptimizeResult(
            x=raw,  # only best x is available
            fun=problem.fun(raw),  # best f(x) value is not available
        )

        return res


# Test

In [12]:
import optimagic as om
import numpy as np
import pyensmallen as pye
from optimagic.optimizers.pyensmallen_optimizers import EnsmallenLBFGS

x0 = np.arange(5)

def sphere(x):
    return x@x

def sphere_grad(x):
    return 2*x

def sphere_and_grad(x):
    return x@x, 2*x


In [13]:
res = om.minimize(
    fun = sphere,
    params = x0,
    algorithm = 'scipy_lbfgsb',
    bounds = om.Bounds(lower = np.zeros_like(x0))

)
res.params.round(3)

array([0., 0., 0., 0., 0.])

In [ ]:
algo = EnsmallenLBFGS(stopping_maxiter=100,)
# algo = om.algorithms.EnsmallenLBFGS()
res = om.minimize(
    fun = sphere,
    # jac = sphere_grad,
    params = x0,
    algorithm = algo,
    fun_and_jac = sphere_and_grad
)
res.params.round(3)
# om.criterion_plot(res)

array([0., 0., 0., 0., 0.])

# Rosenbrock function

In [6]:
def rosenbrock(x, a=2):
    return np.sum((a - x[:-1]) ** 2 + 100.0 * (x[1:] - x[:-1] ** 2) ** 2)


def rosenbrock_gradient(x, a=2):
    grad = np.zeros_like(x)
    # Gradient for the first element
    grad[0] = -2 * (a - x[0]) - 400 * x[0] * (x[1] - x[0] ** 2)
    # Gradient for the middle elements
    grad[1:-1] = (
        -2 * (a - x[1:-1])
        + 200 * (x[1:-1] - x[:-2] ** 2)
        - 400 * x[1:-1] * (x[2:] - x[1:-1] ** 2)
    )
    # Gradient for the last element
    grad[-1] = 200 * (x[-1] - x[-2] ** 2)
    return grad


def objective_function(x, grad):
    grad[:] = rosenbrock_gradient(x)
    return rosenbrock(x)



x0 = np.array([-1.2, 1.0])

optimizer = pye.L_BFGS()
res= optimizer.optimize(objective_function, x0)
type(res)
optimizer.maxIterations

10000

# Other

In [87]:
algo = om.algos.ensmallen_lbfgs(stopping_maxiter=2)
res = om.minimize(
    fun = lambda x: x@x,
    params = np.arange(5),
    algorithm=algo,
)
expected= np.array([ 0.000000100e+00, -2.197921242e-07, -4.39584284e-07, -1.268622263e-07,-6.57024213e-09])
print(np.allclose(res.params, expected))
res.x


False


array([ 0.00000000e+00, -2.19792142e-07, -4.39584284e-07, -1.26862263e-07,
       -6.57024213e-09])